# The Battle of Neighborhoods

In [143]:
import pandas as pd # Data Analysis
import numpy as np # Handles Data in a Vectorized Manner
import requests #handles requests
import folium # Plotting
from geopy.geocoders import Nominatim # address to Latitude and Longitude
from geopy.extra.rate_limiter import RateLimiter # reduces the "Too Many Requests Error"
from bs4 import BeautifulSoup # web scraping


print('Modules Imported!')

Modules Imported!


In [144]:
# Web Sources for scraping

NYC_url = 'https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm'
LA_url = 'http://www.laalmanac.com/communications/cm02_communities.php'
CHI_url = 'https://www.seechicagorealestate.com/chicago-zip-codes-by-neighborhood.php'

## Web Scraping and setting up DataFrames

New York City

In [145]:
source1 = requests.get(NYC_url).text
soup1 = BeautifulSoup(source1,'lxml')
table1 = soup1.find('table')

readme_html1 = pd.read_html(str(table1))
NYC_df = pd.DataFrame(readme_html1[0])
print(NYC_df.shape, '\n', NYC_df.head())

(42, 3) 
   Borough                Neighborhood                   ZIP Codes
0   Bronx               Central Bronx         10453, 10457, 10460
1   Bronx      Bronx Park and Fordham         10458, 10467, 10468
2   Bronx  High Bridge and Morrisania         10451, 10452, 10456
3   Bronx  Hunts Point and Mott Haven  10454, 10455, 10459, 10474
4   Bronx   Kingsbridge and Riverdale                10463, 10471


Los Angeles 

In [146]:
source2 = requests.get(LA_url).text
soup2 = BeautifulSoup(source2, 'lxml')
table2 = soup2.find('table')

readme_html2 = pd.read_html(str(table2))
LA_df = pd.DataFrame(readme_html2[0])
print(LA_df.shape, '\n', LA_df.head())

(643, 2) 
             City/Community   Zip Code(s)
0                    Acton         93510
1             Agoura Hills         91301
2  Agoura Hills (PO Boxes)         91376
3               Agua Dulce         91390
4                 Alhambra  91801, 91803


Chicago

In [147]:
source3 = requests.get(CHI_url).text
soup3 = BeautifulSoup(source3, 'lxml')
table3 = soup3.find('table')

readme_html3 = pd.read_html(str(table3))
CHI_df = pd.DataFrame(readme_html3[0])
print(CHI_df.shape, '\n', CHI_df.head())

(199, 2) 
                     0             1
0            Downtown      Zip Code
1  Cathedral District         60611
2     Central Station         60605
3       Dearborn Park         60605
4          Gold Coast  60610, 60611


Renaming columns to be consistent through each DataFrame

In [148]:
NYC_df.rename(columns={'ZIP Codes': 'zip_codes'}, inplace=True)
LA_df.rename(columns={'City/Community': 'Neighborhood', 'Zip Code(s)': 'zip_codes'}, inplace=True)
CHI_df.columns = ['Neighborhood','zip_codes']

print(list(NYC_df.columns))
print(list(LA_df.columns))
print(list(CHI_df.columns))

['Borough', 'Neighborhood', 'zip_codes']
['Neighborhood', 'zip_codes']
['Neighborhood', 'zip_codes']


## Cleaning DataFrames

In [149]:
#Checking NA and NULLs in each DataFrame

print(NYC_df.isna().sum())
print(NYC_df.isnull().sum())
print('\n')
print(LA_df.isna().sum())
print(LA_df.isnull().sum())
print('\n')
print(CHI_df.isna().sum())
print(CHI_df.isnull().sum())

Borough         0
Neighborhood    0
zip_codes       0
dtype: int64
Borough         0
Neighborhood    0
zip_codes       0
dtype: int64


Neighborhood    0
zip_codes       0
dtype: int64
Neighborhood    0
zip_codes       0
dtype: int64


Neighborhood    7
zip_codes       7
dtype: int64
Neighborhood    7
zip_codes       7
dtype: int64


In [150]:
#Investigating the CHI DataFrame

#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)
#CHI_df

In [151]:
CHI_df.dropna(inplace=True)
print(CHI_df.isna().sum())
print(CHI_df.isnull().sum())

Neighborhood    0
zip_codes       0
dtype: int64
Neighborhood    0
zip_codes       0
dtype: int64


### Exploding DataFrame by its ZIP Code

since there are multiple zip codes for each neighborhood, I need to separate them so the the geo locator can locate each zipcode

In [152]:
new_df1 = pd.DataFrame(NYC_df.zip_codes.str.split(',').tolist(), index= NYC_df.Neighborhood).stack()
new_df1 = new_df1.reset_index([0, 'Neighborhood'])
NYC_df2 = new_df1
NYC_df2.columns = ['Neighborhood','zip_codes']
NYC_df2

,Neighborhood,zip_codes
0,Central Bronx,10453
1,Central Bronx,10457
2,Central Bronx,10460
3,Bronx Park and Fordham,10458
4,Bronx Park and Fordham,10467
...,...,...
173,South Shore,10312
174,Stapleton and St. George,10301
175,Stapleton and St. George,10304
176,Stapleton and St. George,10305


In [153]:
new_df2 = pd.DataFrame(LA_df.zip_codes.str.split(',').tolist(), index= LA_df.Neighborhood).stack()
new_df2 = new_df2.reset_index([0, 'Neighborhood'])
LA_df2 = new_df2
LA_df2.columns = ['Neighborhood','zip_codes']
LA_df2

,Neighborhood,zip_codes
0,Acton,93510
1,Agoura Hills,91301
2,Agoura Hills (PO Boxes),91376
3,Agua Dulce,91390
4,Alhambra,91801
...,...,...
975,Woodland Hills (Los Angeles),91364
976,Woodland Hills (Los Angeles),91367
977,Woodland Hills (PO Boxes) (Los Angeles),91365
978,Woodland Hills (PO Boxes) (Los Angeles),91372


In [154]:
new_df3 = pd.DataFrame(CHI_df.zip_codes.str.split(',').tolist(), index= CHI_df.Neighborhood).stack()
new_df3 = new_df3.reset_index([0, 'Neighborhood'])
new_df3.drop(new_df3.index[0], inplace=True)
new_df3.reset_index(inplace=True)
CHI_df2 = new_df3
CHI_df2.columns = ['old index' , 'Neighborhood','zip_codes']
CHI_df2.drop(['old index'], axis=1, inplace= True)
CHI_df2

,Neighborhood,zip_codes
0,Cathedral District,60611
1,Central Station,60605
2,Dearborn Park,60605
3,Gold Coast,60610
4,Gold Coast,60611
...,...,...
355,Washington Park,60609
356,Washington Park,60615
357,Washington Park,60621
358,Washington Park,60637


## Finding Geo Coordinates for each ZIP Code

In [155]:
#geolocator = Nominatim(user_agent='google_agent')
#geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
#NYC_df2['location'] = NYC_df2['zip_codes'].apply(geocode)

#NYC_df2['point'] = NYC_df2['location'].apply(lambda loc: tuple(loc.point) if loc else None)

#print('DONE!')

In [156]:
#LA_df2['location'] = LA_df2['zip_codes'].apply(geocode)

#LA_df2['point'] = LA_df2['location'].apply(lambda loc: tuple(loc.point) if loc else None)

#print('DONE!')

In [157]:
#CHI_df2['location'] = CHI_df2['zip_codes'].apply(geocode)

#CHI_df2['point'] = CHI_df2['location'].apply(lambda loc: tuple(loc.point) if loc else None)

#print('DONE!')

Saving results so that I dont have to re run the code each time. Takes a while to find to reduce the "Too Many Requests Error"

In [158]:
#NYC_df2.to_csv('NYC_df2', index=False)
#LA_df2.to_csv('LA_df2', index=False)
#CHI_df2.to_csv('CHI_df2', index=False)

#print('DONE!')

In [159]:
NYC_df2 = pd.read_csv('NYC_df2')
LA_df2 = pd.read_csv('LA_df2')
CHI_df2 = pd.read_csv('CHI_df2')

Cleaning DataFrames Again. Because of there were coordinates and location outside of the US. I believe this happened because of the limitations of the Nominatim Function.

In [160]:
NYC_df2 = NYC_df2[NYC_df2.location.str.contains('New York')]
NYC_df2

,Neighborhood,zip_codes,location,point
0,Central Bronx,10453,"The Bronx, Bronx County, New York, 10453, Unit...","(40.85234385146179, -73.91196955537043, 0.0)"
1,Central Bronx,10457,"The Bronx, Bronx County, New York, 10457, Unit...","(40.84796417510888, -73.89770955547682, 0.0)"
3,Bronx Park and Fordham,10458,"The Bronx, Bronx County, New York, 10458, Unit...","(40.86156201215534, -73.88877720559567, 0.0)"
4,Bronx Park and Fordham,10467,"The Bronx, Bronx County, New York, 10467, Unit...","(40.87452493805371, -73.86794642958414, 0.0)"
5,Bronx Park and Fordham,10468,"The Bronx, Bronx County, New York, 10468, Unit...","(40.8710838, -73.8941052, 0.0)"
...,...,...,...,...
173,South Shore,10312,"Staten Island, Richmond County, New York, 1031...","(40.5436525, -74.16830055, 0.0)"
174,Stapleton and St. George,10301,"Staten Island, Richmond County, New York, 1030...","(40.63979875, -74.07574570091533, 0.0)"
175,Stapleton and St. George,10304,"Todt Hill, Staten Island, Richmond County, New...","(40.606031, -74.086625, 0.0)"
176,Stapleton and St. George,10305,"Todt Hill, Staten Island, Richmond County, New...","(40.609344, -74.076293, 0.0)"


In [161]:
CHI_df2 = CHI_df2[CHI_df2.location.str.contains('Chicago')]
CHI_df2

,Neighborhood,zip_codes,location,point
0,Cathedral District,60611,"Near North Side, Chicago, Cook County, Illinoi...","(41.89513283566048, -87.62281879221906, 0.0)"
1,Central Station,60605,"Near South Side, Chicago, Cook County, Illinoi...","(41.8658506, -87.6099423, 0.0)"
2,Dearborn Park,60605,"Near South Side, Chicago, Cook County, Illinoi...","(41.8658506, -87.6099423, 0.0)"
3,Gold Coast,60610,"Near North Side, Chicago, Cook County, Illinoi...","(41.8884984, -87.6292815, 0.0)"
4,Gold Coast,60611,"Near North Side, Chicago, Cook County, Illinoi...","(41.89513283566048, -87.62281879221906, 0.0)"
...,...,...,...,...
355,Washington Park,60609,"Bridgeport, Chicago, Cook County, Illinois, 60...","(41.8249226, -87.6381346, 0.0)"
356,Washington Park,60615,"Hyde Park, Chicago, Cook County, Illinois, 606...","(41.80110300957798, -87.59432360924957, 0.0)"
357,Washington Park,60621,"Englewood, Chicago, Cook County, Illinois, 606...","(41.7775939545224, -87.63663948181107, 0.0)"
358,Washington Park,60637,"Hyde Park, Chicago, Cook County, Illinois, 606...","(41.7884978, -87.5793199, 0.0)"


Due to NA/NULL values the same operation of filtering the data didnt work. So I had to drop them again. Possibly due to the limitations of Nominatim.


In [162]:
print(LA_df2.isna().sum())
print(LA_df2.isnull().sum())

Neighborhood     0
zip_codes        0
location        88
point           88
dtype: int64
Neighborhood     0
zip_codes        0
location        88
point           88
dtype: int64


In [163]:
LA_df2.dropna(inplace=True)
print(LA_df2.isna().sum())
print(LA_df2.isnull().sum())

Neighborhood    0
zip_codes       0
location        0
point           0
dtype: int64
Neighborhood    0
zip_codes       0
location        0
point           0
dtype: int64


In [164]:
LA_df2 = LA_df2[LA_df2.location.str.contains('Los Angeles')]
LA_df2


,Neighborhood,zip_codes,location,point
0,Acton,93510,"Los Angeles County, California, 93510, United ...","(34.46103834645381, -118.21138549680546, 0.0)"
1,Agoura Hills,91301,"Malibu Junction, Agoura Hills, Los Angeles Cou...","(34.15295555, -118.75932802244274, 0.0)"
5,Alhambra,91803,"Alhambra, Los Angeles County, California, 9180...","(34.07170388325506, -118.1458605938974, 0.0)"
7,Alhambra (PO Boxes),91802,"Alhambra, Los Angeles County, California, 9180...","(34.0927825, -118.1263605, 0.0)"
10,Altadena,91001,"Linda Vista, Altadena, Los Angeles County, Cal...","(34.1900323, -118.1325732, 0.0)"
...,...,...,...,...
956,Wilmington (Los Angeles),90744,"Thenard, Los Angeles, Los Angeles County, Cali...","(33.7933964, -118.2400379, 0.0)"
958,Wilshire Center (Los Angeles),90004,"Los Angeles, Los Angeles County, California, 9...","(34.071844381995426, -118.3023158073664, 0.0)"
966,Wilshire-La Brea (Los Angeles),90036,"Los Angeles, Los Angeles County, California, 9...","(34.06737818754923, -118.352266061733, 0.0)"
968,Windsor Hills (Los Angeles),90043,"Los Angeles, Los Angeles County, California, 9...","(33.992140810110676, -118.33125048227424, 0.0)"


In [165]:
NYC_df2.drop_duplicates(subset= 'zip_codes', keep= False, inplace=True)
LA_df2.drop_duplicates(subset= 'zip_codes', keep= False, inplace=True)
CHI_df2.drop_duplicates(subset= 'zip_codes', keep= False, inplace=True)

print('Duplicates Removed!')

Duplicates Removed!


Setting up for column split insteads of "point" it'll be  "Latitude" & "Longitude"

In [166]:
NYC_df2=NYC_df2.replace('\(','',regex=True).astype(str)
NYC_df2=NYC_df2.replace('\)','',regex=True).astype(str)
NYC_df2=NYC_df2.replace('\, 0.0','',regex=True).astype(str)

LA_df2=LA_df2.replace('\(','',regex=True).astype(str)
LA_df2=LA_df2.replace('\)','',regex=True).astype(str)
LA_df2=LA_df2.replace('\, 0.0','',regex=True).astype(str)

CHI_df2=CHI_df2.replace('\(','',regex=True).astype(str)
CHI_df2=CHI_df2.replace('\)','',regex=True).astype(str)
CHI_df2=CHI_df2.replace('\, 0.0','',regex=True).astype(str)

In [167]:
NYC_df2.reset_index(inplace=True)
LA_df2.reset_index(inplace=True)
CHI_df2.reset_index(inplace=True)

Made new DataFrame with Lat and Long values

In [168]:
NYC_df2.drop(['index'], axis=1, inplace= True)
LA_df2.drop(['index'], axis=1, inplace= True)
CHI_df2.drop(['index'], axis=1, inplace= True)

In [169]:
split_df1 = NYC_df2['point'].str.split(",")
data1 = split_df1.to_list()
names1 = ['Latitude', 'Longitude']
M1_df = pd.DataFrame(data1, columns=names1)

In [170]:
split_df2 = LA_df2['point'].str.split(",")
data2 = split_df2.to_list()
names2 = ['Latitude', 'Longitude']
M2_df = pd.DataFrame(data2, columns=names2)

In [171]:
split_df3 = CHI_df2['point'].str.split(",")
data3 = split_df3.to_list()
names3 = ['Latitude', 'Longitude']
M3_df = pd.DataFrame(data3, columns=names3)

In [172]:
NYC_df3 = pd.concat([NYC_df2, M1_df], axis=1, join='inner')
LA_df3 = pd.concat([LA_df2, M2_df], axis=1, join='inner')
CHI_df3 = pd.concat([CHI_df2, M3_df], axis=1, join='inner')

print('Concatenated objects')

Concatenated objects


In [173]:
NYC_df3.drop(['point'], axis=1, inplace= True)
LA_df3.drop(['point'], axis=1, inplace= True)
CHI_df3.drop(['point'], axis=1, inplace= True)

print("Columns Dropped!")

Columns Dropped!
